In [1]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [4]:
total_gene_dir = "/data5/galaxy/project/promoter_TF_enrich/data/total_gene/gene_bed"
total_gene_list = glob.glob("%s/*.bed" % total_gene_dir)
#
m6a_gene_dir = "/data5/galaxy/project/promoter_TF_enrich/data/m6a_gene/gene_bed"
m6a_gene_list = glob.glob("%s/*.bed" % m6a_gene_dir)
#
CpG_high_gene = "/data5/galaxy/project/CpG_m6a_motif/fasta_seq/high_CpG.bed"
CpG_low_gene = "/data5/galaxy/project/CpG_m6a_motif/fasta_seq/low_CpG.bed"
high_CpG_genes = list(set(pd.read_table(CpG_high_gene, sep="\t", header=None).iloc[:, 3]))
low_CpG_genes = list(set(pd.read_table(CpG_low_gene, sep="\t", header=None).iloc[:, 3]))

In [5]:
tissue_list, propor_high_m6a, propor_low_m6a = [], [], []
for total_gene in total_gene_list:
    tissue = os.path.basename(total_gene).split(".")[0]
    print(tissue)
    high_CpG, low_CpG = class_CpG(total_gene)
    m6a_genes = get_m6a_genes(total_gene)
    highCpG_m6a, highGpG_nonm6a = [x for x in high_CpG if x in m6a_genes], [x for x in high_CpG if x not in m6a_genes]
    lowCpG_m6a, lowGpG_nonm6a = [x for x in low_CpG if x in m6a_genes], [x for x in low_CpG if x not in m6a_genes]
    highCpG_proportation = len(highCpG_m6a) / len(high_CpG)
    lowCpG_proportation = len(lowCpG_m6a) / len(low_CpG)
    #
    tissue_list.append(tissue)
    propor_high_m6a.append(highCpG_proportation)
    propor_low_m6a.append(lowCpG_proportation)
    print(highCpG_proportation, lowCpG_proportation)

liver
0.7790761081249513 0.5064282844515869
stomach
0.7260907713169019 0.47473637961335674
placenta
0.8291283502318636 0.5580578512396694
lung
0.7458082885162923 0.4737736249734551
heart
0.7711248757360251 0.4349442379182156
brain
0.7823275862068966 0.39058669717274064
muscle
0.7939514524472742 0.5418933623503809
kidney
0.8608695652173913 0.5805587447378492


In [14]:
# https://pythonfordatascience.org/paired-samples-t-test-python/
df = pd.DataFrame({"high_CpG": propor_high_m6a, "low_CpG": propor_low_m6a})
df["difference"] = df.high_CpG - df.low_CpG
# df.head()
### normality distribution test
# df.difference.plot(kind="hist", title="histogram plot for difference proportation of m6a")
# stats.probplot(df.difference, plot=plt)
stats.shapiro(df.difference)
# paired t-test
statistic, pvalue = stats.ttest_rel(propor_high_m6a, propor_low_m6a)
print(statistic, pvalue)

(0.7773181796073914, 0.016288572922348976)

16.955673772950885 6.082164587780182e-07


In [32]:
df["i"] = df.index
df.columns = ["m6a propor-high CpG", "m6a propor-low CpG", "difference", "i"]
df_a = df.copy()[["m6a propor-high CpG", "m6a propor-low CpG", "i"]]
# df_a.head()
df_long = pd.wide_to_long(df_a, "m6a propor", sep="-", suffix='(high CpG|low CpG)', i="i", j="CpG_type")
df_long.head()
df_long.to_csv("/data5/galaxy/project/CpG_proporation-m6aGene/plot_data.txt", sep="\t")

,,m6a propor
i,CpG_type,
0,high CpG,0.779076
1,high CpG,0.726091
2,high CpG,0.829128
3,high CpG,0.745808
4,high CpG,0.771125


In [2]:
#### class GpG high && low
def class_CpG(total_gene):
    total_genes = list(set(pd.read_table(total_gene, sep="\t", header=None).iloc[:, 3]))
    high_CpG = [gene for gene in total_genes if gene in high_CpG_genes]
    low_CpG = [gene for gene in total_genes if gene in low_CpG_genes]
    return high_CpG, low_CpG

def get_m6a_genes(total_gene):
    total_genes = list(set(pd.read_table(total_gene, sep="\t", header=None).iloc[:, 3]))
    base_name = os.path.basename(total_gene)
    m6a_gene = os.path.join(m6a_gene_dir, base_name)
    try:
        m6a_genes = list(set(pd.read_table(m6a_gene, sep="\t", header=None).iloc[:, 3]))
    except FileExistsError:
        print("The basenames of (%s and %s) aren't same!" % (total_gene, m6a_gene))
    return m6a_genes